In [ ]:
# Can we find parent/child word relationships faster
# One way to achieve this is to use numba which pre-compiles code

In [ ]:
# standard libraries
from time import perf_counter_ns
import time

In [ ]:

# external libraries
import pandas as pd
import numpy as np

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *


# Load input data

In [ ]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)


In [ ]:
wg_df.shape

# let's process 1000 rows using a single lookup on the full wchar_matrix

In [ ]:
n_possible_anagrams = int(1e6)

In [ ]:
f"{n_possible_anagrams :,}"

In [ ]:
import numba

In [ ]:
arr = np.array([[True, False, True], [True, True, True]])

In [ ]:
arr.shape

In [ ]:
for row in arr:
    print(row)

In [ ]:
@numba.njit
def all_axis_0(arr):
    outcome = np.zeros(shape = arr.shape[0],dtype=np.bool)
    row_count = 0
    for row in arr:        
        if row.min() >= 0:
            outcome[row_count] = 1
        row_count += 1
    return outcome

In [ ]:
@numba.njit
def all_axis_1(arr):
    arr_t = arr.T
    outcome = np.zeros(shape = arr_t.shape[0],dtype=np.bool)
    row_count = 0
    for row in arr_t:        
        if row.min() >= 0:
            outcome[row_count] = 1
        row_count += 1
    return outcome

In [ ]:
all_axis_0(arr=arr)

In [ ]:
all_axis_1(arr=arr)

In [ ]:
# try this out with numba
@numba.njit()
def do_something_clever(wchar_matrix, word_group_id_list, n_sample_size = 10):
    n_possible_anagrams = 1000000
    sample_wg_id = np.random.randint(0, 215842, size=n_sample_size)

    # create an output object
    output_list = np.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype=np.int64)

    row_count = 0
    anagram_pair_count = 0

    for i_wg_id, wg_id in enumerate(sample_wg_id):
        # identify parent words
        outcome = wchar_matrix - wchar_matrix[wg_id, ]
        
        # compute the score by finding where rows, across all columns, are GTE 0
        outcome_indices = all_axis_0(arr=outcome)
        #outcome_indices = np.all(outcome >= 0, 1)    

        n_from_words = outcome_indices.sum()

        if n_from_words >= 1:
            # extract anagrams based on index values    
            outcome_word_id_list = word_group_id_list[outcome_indices]    

            # we have matches
            # the focal word
            curr_output_list = np.zeros(shape=(n_from_words, 2), dtype=np.int64)

            # update the output list with the word_id_list - these are from/parent words
            curr_output_list[:, 0] = outcome_word_id_list

            # update with the word_id - this is the to/child word
            curr_output_list[:, 1] = wg_id

            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words

            output_list[anagram_pair_count:new_anagram_pair_count,
                        :] = curr_output_list

            # n_to_word_counter = collections.Counter(output_list[:, 0])
            #intermediate_to_word_count.update(outcome_word_id_list.tolist())

            # set the anagram pair count
            anagram_pair_count = new_anagram_pair_count
        
        row_count += 1

        if row_count % 100 == 0:
            print(row_count)
    print('truncating list')
    #output_indices = np.all(output_list >= 0, axis=1)
    output_indices = all_axis_0(arr = output_list)
    output_list = output_list[output_indices,]    
    
    return output_list

In [ ]:
output_list = do_something_clever(wchar_matrix=wchar_matrix, word_group_id_list=word_group_id_list,
                                  n_sample_size=1000)

In [ ]:
output_list

# Using the CPU

In [ ]:
# sample 1000 rows
n_samples = 1000
sample_wg_id = wg_df['word_group_id'].sample(n = n_samples, random_state = 42).to_list()

if 746 not in sample_wg_id:
    sample_wg_id.append(746)
    

# create an output object
output_list = np.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype=int)

row_count = 0
anagram_pair_count = 0

for i_wg_id, wg_id in enumerate(sample_wg_id):
    # identify parent words
    outcome = wchar_matrix - wchar_matrix[wg_id, ]
    
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices = np.all(outcome >= 0, axis=1)    

    n_from_words = outcome_indices.sum()

    if n_from_words >= 1:
        # extract anagrams based on index values    
        outcome_word_id_list = word_group_id_list[outcome_indices]    

        # we have matches
        # the focal word
        curr_output_list = np.zeros(shape=(n_from_words, 2), dtype=int)

        # update the output list with the word_id_list - these are from/parent words
        curr_output_list[:, 0] = outcome_word_id_list

        # update with the word_id - this is the to/child word
        curr_output_list[:, 1] = wg_id

        # enumerate the from/parent words
        new_anagram_pair_count = anagram_pair_count + n_from_words

        output_list[anagram_pair_count:new_anagram_pair_count,
                    :] = curr_output_list

        # n_to_word_counter = collections.Counter(output_list[:, 0])
        #intermediate_to_word_count.update(outcome_word_id_list.tolist())

        # set the anagram pair count
        anagram_pair_count = new_anagram_pair_count
    
    row_count += 1

    if row_count % 100 == 0:
        print(row_count)
print('truncating list')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
del output_indices
output_list.shape

In [ ]:
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [ ]:
# the number of from word groups
from_word_counter[746]

# build a selector

In [ ]:
# let's split the matrix, gather the values for each split, and then combine
n_subset_letters = 3
wg_df["letter_selector"] = wg_df["letter_group_ranked"].str[:n_subset_letters]

letter_selector_list = wg_df["letter_selector"].unique()
letter_selector_list.sort()
letter_selector_id_dict = {ls: i_ls for i_ls, ls in enumerate(letter_selector_list)}

wg_df["letter_selector_id"] = wg_df["letter_selector"].map(letter_selector_id_dict)
# here's the thing: I need to be able to identify on a single matrix the rows that match various conditions.
# I can't step through it and create objects at abandon. 
# so, given our wchar_matrix: what are the rows that match to such and such?
# we can add three columns to track this... 

In [ ]:
wg_df.head()

In [ ]:
# load letter ranks
sql = 'select letter, total_letter_rank from letter_count;'

In [ ]:
lr_df = query_db(sql = sql, db_path=rc.DB_PATH, db_name=rc.DB_NAME)

In [ ]:
lr_dict = {l:r for l, r in zip(lr_df['letter'], lr_df['total_letter_rank'])}

In [ ]:
wg_df['n_records'] = int(1)

In [ ]:
col_names = ['letter_selector_id', 'letter_selector', 'n_records']

In [ ]:
ls_df = wg_df[col_names].groupby(col_names[:-1]).agg(ls_count = ('n_records', 'sum')).reset_index()

In [ ]:
ls_df.head()

In [ ]:
ls_df['ls_count'].sum()

In [ ]:
ls_df['ls_count'].describe()

In [ ]:
def get_ls_index(ls:str):
    return [letter_dict[l] for l in ls]

In [ ]:
# this is effectively a column selector
ls_df['ls_index'] = ls_df['letter_selector'].map(get_ls_index)

In [ ]:
ls_df.head()

In [ ]:
ls_df.head()

In [ ]:
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH, db_name=rc.DB_NAME)

In [ ]:
import numba

In [ ]:
@numba.jit(nopython=True)
def get_ls_wchar_matrix(wchar_matrix:np.ndarray, ls_id_index:np.ndarray, 
                        word_group_id_list:np.ndarray):
    outcome_indices = np.all(wchar_matrix[:, ls_id_index] >= 1, 1)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list = word_group_id_list[outcome_indices]

    return ls_wchar_matrix, temp_wg_id_list
    


In [ ]:
# create the output list
output_list = np.full(shape = (n_possible_anagrams, 2), fill_value=-1)
output_time_list = []

# start counting
anagram_pair_count = 0

#for ls_id_index in range(0, 10):
for ls_row_id, ls_row in ls_df.iloc[:None].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information
    ls_id = ls_row['letter_selector_id']
    ls_id_index = np.array(ls_row['ls_index'])    

    ##
    # BUILD A COLUMN SELECTOR
    ##
    # make sure that only values GTE 0 are selected so that the right number of
    # columns are return.
    #curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selector
    ##
     
    ls_wchar_matrix, temp_wg_id_list = get_ls_wchar_matrix(wchar_matrix = wchar_matrix, 
                                                           ls_id_index = ls_id_index,
                                                           word_group_id_list = word_group_id_list)    

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    n_search_space = temp_wg_id_list.shape[0]
        
    #def my_func(row):
    #    return temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[row, :]) >= 0, axis = 1)]

    #for ii in range(0, ls_wchar_matrix.shape[0]):    
    #for i_curr_wg_id, curr_wg_id in enumerate(temp_wg_id_list):
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'bro'    
    curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_numpy()
    # n_lookups = curr_wg_id_list.shape[0]
    # n_search_space >= n_lookups, always. 
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):
    
        
        #temp_wg_id = wg_id_dict[curr_wg_id]
        temp_wg_id = np.where(temp_wg_id_list == curr_wg_id)[0][0]
        #print(curr_wg_id, temp_wg_id)

        #outcome_word_id_list = my_func(row = temp_wg_id)
        outcome_word_id_list = temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]
                
        n_from_words = outcome_word_id_list.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
            #print(outcome_word_id_list.shape)
            
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            #print(anagram_pair_count, new_anagram_pair_count)

            output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, curr_time])

In [ ]:
def build_timing_and_output_objects(output_time_list:list, ls_df:pd.DataFrame):
    
    col_names =['letter_selector_id', 'n_search_space', 'curr_time']
    time_df = pd.DataFrame(data = output_time_list, columns=col_names)
    get_hms(seconds = time_df['curr_time'].sum(),round_seconds_digits=4)
    # join in the other information
    time_df = pd.merge(left = time_df, right = ls_df)

    time_df['avg_lookup_time'] = time_df['curr_time'] / (time_df['ls_count'] * time_df['n_search_space'])

    return time_df   

In [ ]:
time_df = build_timing_and_output_objects(output_time_list=output_time_list,
                                          ls_df = ls_df)

In [ ]:
time_df.head()

In [ ]:
time_df.tail()

In [ ]:
time_df['n_search_space'].describe()

In [ ]:
time_df['ls_count'].sum()

In [ ]:
time_df['avg_lookup_time'].describe()

In [ ]:
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
output_list.shape

In [ ]:
print('...building output counters...')
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [ ]:
# check to see if the word 'acanthology', with word_group_id 746 is in the counter
print(from_word_counter[746])
print(to_word_counter[746])